In [1]:
%matplotlib inline

In [2]:
import matplotlib as plt
from PIL import ImageFont, ImageDraw, Image
import random as rd
from IPython.display import display
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
from skimage import io, transform
import torchvision


In [3]:
path_image = "C:\\Users\\Dinmuhametov Salavat\\Desktop\\ak_bars\\"

In [4]:
xl, xr, yu, yd = 0.06, 0.312, 0.2, 0.48

In [5]:
def draw_rectangle(image):
    image = image.copy()
    x, y = image.size
    draw = ImageDraw.Draw(image)
    draw.polygon([(xl*x, yu*y), (xr*x, yu*y), (xr*x, yd*y), (xl*x, yd*y)])
    return image

In [13]:
def crop_numbers(image):
    image = image.copy()
    x, y = image.size
    dx, dy = int(x*(xr-xl)/4), int(y*(yd-yu)/4)
    num = []
    for i in range(4):
        num.append([])
        for j in range(4):
            num[i].append(image.crop((xl*x+dx*j, y*yu+dy*i,
                                      xl*x+dx*(j+1), y*yu+dy*(i+1))))
            num[i][-1] = num[i][-1].resize((244, 244),Image.ANTIALIAS)
#             display(num[i][j])
    return num

In [9]:
im = Image.open(path_image+'card_normalized_'+'793'+'.jpg')

In [7]:
# crop_numbers(draw_rectangle(draw_numbers(im)[0]))

In [9]:
# im1 = Image.open(path_image+'card.png')
# crop_numbers(im1)

In [7]:
def draw_numbers(image):
    image = image.copy()
    x, y = image.size
    draw = ImageDraw.Draw(image)
    fnt = ImageFont.truetype("C:\\Users\\Dinmuhametov Salavat\\Desktop\\ak_bars\\CREDC___.ttf", int(y*0.0503125), encoding="unic")
    r_num = ''
    for i in range(4):
        new_rnum = str(rd.randint(1000, 9999))
        draw.text((xl*x , y*yu+ (y*(yd-yu)/4)*i), new_rnum, font = fnt, fill=(255,255,255,255)) 
        r_num += new_rnum
        
    return image, r_num

In [17]:
index = 0
dataset = pd.DataFrame(data={'path':[],'label':[]})
for j in range(63):
    image, labels = draw_numbers(im)
    digits = crop_numbers(image)
    for i in range(16):
        digit_path = path_image+'train\\'+ str(labels[i]) + '\\' +str(index)+'.jpg'
#         new_dataset = pd.DataFrame(data={'path':['dataset\\'+str(index)+'.jpg'],'label':[labels[i]]})
#         dataset = dataset.append(new_dataset, ignore_index=True)
        digits[i//4][i%4].save(digit_path)
        index+=1
        
dataset.to_csv(path_image+'train\\data.csv')

In [ ]:
# ds = pd.read_csv(path_image+'dataset\\data.csv')
# ds = ds.drop('Unnamed: 0', axis=1)

In [ ]:
# ds.head()

,path,label
0,dataset\0.jpg,4
1,dataset\1.jpg,0
2,dataset\2.jpg,1
3,dataset\3.jpg,0
4,dataset\4.jpg,2


In [ ]:
class DigitsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.landmarks_frame = self.landmarks_frame.drop('Unnamed: 0', axis=1)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        label = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        label = label.astype('int')
        sample = {'data': image, 'target': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['data'], sample['target']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'data': torch.from_numpy(image),
                'target': torch.from_numpy(landmarks)}

In [ ]:
class Normalaze(object):
    
    def __call__(self, sample):
        image, landmarks = sample['data'], sample['target']
        image = torchvision.transforms.functional.normalize(image, mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        return {'data': image,
                'target': landmarks}

In [ ]:
# dt = DigitsDataset(csv_file=path_image+'dataset\\data.csv',root_dir=path_image)


In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, shit in enumerate(train_loader):
        data, target = shit['data'], shit['target']
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [ ]:
# def main():
#     # Training settings
#     parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
#     parser.add_argument('--batch-size', type=int, default=64, metavar='N',
#                         help='input batch size for training (default: 64)')
#     parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                         help='input batch size for testing (default: 1000)')
#     parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                         help='number of epochs to train (default: 10)')
#     parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                         help='learning rate (default: 0.01)')
#     parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
#                         help='SGD momentum (default: 0.5)')
#     parser.add_argument('--no-cuda', action='store_true', default=False,
#                         help='disables CUDA training')
#     parser.add_argument('--seed', type=int, default=1, metavar='S',
#                         help='random seed (default: 1)')
#     parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                         help='how many batches to wait before logging training status')
#     args = parser.parse_args()
#     use_cuda = not args.no_cuda and torch.cuda.is_available()

#     torch.manual_seed(args.seed)

#     device = torch.device("cuda" if use_cuda else "cpu")

#     kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
#     train_loader = torch.utils.data.DataLoader(
#         datasets.MNIST('../data', train=True, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor(),
#                            transforms.Normalize((0.1307,), (0.3081,))
#                        ])),
#         batch_size=args.batch_size, shuffle=True, **kwargs)
#     test_loader = torch.utils.data.DataLoader(
#         datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                            transforms.ToTensor(),
#                            transforms.Normalize((0.1307,), (0.3081,))
#                        ])),
#         batch_size=args.test_batch_size, shuffle=True, **kwargs)


#     model = Net().to(device)
#     optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

#     for epoch in range(1, args.epochs + 1):
#         train(args, model, device, train_loader, optimizer, epoch)
#         test(args, model, device, test_loader)


# if __name__ == '__main__':
#     main()

In [ ]:
# args = parser.parse_args()
use_cuda = torch.cuda.is_available()

torch.manual_seed(42)

device = torch.device("cuda" if use_cuda else "cpu")

print('1')

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = DataLoader(DigitsDataset(csv_file=path_image+'dataset\\data.csv',root_dir=path_image, transform=transforms.Compose([
                       ToTensor()
                       ,Normalaze()
                   ])),batch_size=1, shuffle=True, **kwargs)
print('n')
test_loader = DataLoader(DigitsDataset(csv_file=path_image+'test\\data.csv',root_dir=path_image, transform=transforms.Compose([
                       ToTensor(),
                        Normalaze()
                   ])),batch_size=1, shuffle=True, **kwargs)
print('suka rabotay')
# train_loader = DigitsDataset(
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.batch_size, shuffle=True, **kwargs)
# test_loader = DigitsDataset(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

print('a')

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

print('b')

for epoch in range(1, 10 + 1):
    print('blyat')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

1
n
suka rabotay
a
b
blyat


c:\users\dinmuhametov salavat\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
